In [6]:
from nordpool import elspot
import pandas as pd
import numpy as np
import openmeteo_requests
import requests_cache



In [7]:
def fetch_nordpool_data(area: str = "EE"):
    """Fetch Nord Pool electricity prices for a specific area."""
    # Create an instance of the elspot API
    prices_spot = elspot.Prices()
    
    # Fetch hourly prices for today
    data = prices_spot.hourly(areas=[area])

    # Extract data for the specific area
    area_data = data["areas"].get(area, {})
    print(f"Prices for {area}: {area_data}")
    return area_data

if __name__ == "__main__":
    estonia_prices = fetch_nordpool_data("EE")

Prices for EE: {'values': [{'start': datetime.datetime(2025, 1, 9, 23, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 0, 0, tzinfo=tzutc()), 'value': 79.76}, {'start': datetime.datetime(2025, 1, 10, 0, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 1, 0, tzinfo=tzutc()), 'value': 79.74}, {'start': datetime.datetime(2025, 1, 10, 1, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 2, 0, tzinfo=tzutc()), 'value': 54.95}, {'start': datetime.datetime(2025, 1, 10, 2, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 3, 0, tzinfo=tzutc()), 'value': 50.94}, {'start': datetime.datetime(2025, 1, 10, 3, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 4, 0, tzinfo=tzutc()), 'value': 66.18}, {'start': datetime.datetime(2025, 1, 10, 4, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 5, 0, tzinfo=tzutc()), 'value': 75.09}, {'start': datetime.datetime(2025, 1, 10, 5, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 6, 0, tzinfo=tzutc(

In [8]:
df = pd.DataFrame(estonia_prices["values"])
df

,start,end,value
0,2025-01-09 23:00:00+00:00,2025-01-10 00:00:00+00:00,79.76
1,2025-01-10 00:00:00+00:00,2025-01-10 01:00:00+00:00,79.74
2,2025-01-10 01:00:00+00:00,2025-01-10 02:00:00+00:00,54.95
3,2025-01-10 02:00:00+00:00,2025-01-10 03:00:00+00:00,50.94
4,2025-01-10 03:00:00+00:00,2025-01-10 04:00:00+00:00,66.18
5,2025-01-10 04:00:00+00:00,2025-01-10 05:00:00+00:00,75.09
6,2025-01-10 05:00:00+00:00,2025-01-10 06:00:00+00:00,101.21
7,2025-01-10 06:00:00+00:00,2025-01-10 07:00:00+00:00,125.63
8,2025-01-10 07:00:00+00:00,2025-01-10 08:00:00+00:00,141.30
9,2025-01-10 08:00:00+00:00,2025-01-10 09:00:00+00:00,137.30


### Get weather data

In [9]:
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [12]:
import requests
import pandas as pd

# Coordinates for Tallinn, Estonia
latitude = 59.437
longitude = 24.7535

url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 59.4165,
	"longitude": 24.7994,
	"hourly": ["temperature_2m", "wind_speed_10m", "direct_radiation", "diffuse_radiation"],
	"forecast_days": 1
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_direct_radiation = hourly.Variables(2).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["direct_radiation"] = hourly_direct_radiation
hourly_data["diffuse_radiation"] = hourly_diffuse_radiation

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

Coordinates 59.41769027709961°N 24.804061889648438°E
Elevation 47.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m,wind_speed_10m,direct_radiation,diffuse_radiation
0,2025-01-09 00:00:00+00:00,0.75,25.919998,0.000000,0.000000
1,2025-01-09 01:00:00+00:00,0.60,26.280001,0.000000,0.000000
2,2025-01-09 02:00:00+00:00,0.80,28.080000,0.000000,0.000000
3,2025-01-09 03:00:00+00:00,0.85,25.919998,0.000000,0.000000
4,2025-01-09 04:00:00+00:00,1.05,22.680000,0.000000,0.000000
5,2025-01-09 05:00:00+00:00,1.25,18.719999,0.000000,0.000000
6,2025-01-09 06:00:00+00:00,1.50,19.080000,0.000000,0.000000
7,2025-01-09 07:00:00+00:00,1.45,14.400000,0.000000,0.000000
8,2025-01-09 08:00:00+00:00,1.55,18.719999,-0.143214,2.143214
9,2025-01-09 09:00:00+00:00,1.80,16.559999,1.476683,25.523317
